In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

# Use a service account
cred = credentials.Certificate('./key.json')
firebase_admin.initialize_app(cred)

In [5]:
import subprocess

command = "mkdir data"
subprocess.run(command, shell=True, executable="/bin/bash")

CompletedProcess(args='mkdir data', returncode=0)

In [6]:
import random
def get_all_documents(date=None):
    db = firestore.client()
    ref = db.collection('toi_news_2024-10-22')
    docs = ref.get()
    for doc in docs:
        if random.uniform(0,1) < 0.9:
            continue
        with open(f"./data/{doc.id}.txt", 'w') as file:
            file.write(doc.to_dict()['content'])
            
get_all_documents()

In [7]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader

In [8]:
loader = DirectoryLoader('./data/', glob="*.txt", loader_cls=TextLoader)
news_articles = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [50]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 25
)

#Create a split of the document using the text splitter
news_articles_split = text_splitter.split_documents(news_articles)

In [51]:
print(f"Total news documents before split: {len(news_articles)}")
print(f"Total news documents after split: {len(news_articles_split)}")

Total news documents before split: 51
Total news documents after split: 628


In [52]:
news_articles_split[:2]

[Document(metadata={'source': 'data/article_114433474.txt'}, page_content='Confidence will rise through the day, prompting bold decisions that could lead to significant achievements. Efforts in education and career are likely to be fruitful, while financial gains require'),
 Document(metadata={'source': 'data/article_114433474.txt'}, page_content='financial gains require steady work. Relationships remain neutral, favoring communication. Physical activity can enhance well-being amid a tendency toward laziness. Stay focused to seize any')]

In [84]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

In [85]:
embedding_model = OllamaEmbeddings(model="llama3.1")

In [86]:
# chroma_client = chromadb.PersistentClient("./chromadb")

In [87]:
# chroma_client.heartbeat()

In [88]:
# import chromadb.utils.embedding_functions as embedding_functions
# ollama_ef = embedding_functions.OllamaEmbeddingFunction(
#     url="https://localhost:11434/api/embeddings",
#     model_name="llama3",
# )

In [89]:
# embeddings = ollama_ef(["This is my first text to embed"])

In [90]:
# collection = chroma_client.create_collection(name="news_2024_10_22", embedding_function=embedding_model)

In [91]:
# collection = chroma_client.create_collection(name="news_2024_10_22")

In [92]:
# collection.add(documents=news_articles_split, ids=[], )

In [93]:
vector_store = Chroma(
    collection_name="news_2024_10_22",
    embedding_function=embedding_model,
    persist_directory="./chroma_db"
)

/var/folders/wz/cnwy77v95fz65y4_p_vs2l140000gn/T/ipykernel_79458/4166593299.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [39]:
from uuid import uuid4
from tqdm.auto import tqdm
import time

In [40]:
ids = [str(uuid4()) for _ in range(len(news_articles_split))]

In [42]:
for doc in tqdm(news_articles_split):
    tmp = [doc]
    tmp_id = [str(uuid4())]
    vector_store.add_documents(documents=tmp, ids=tmp_id)

100%|███████████████████████████████████████| 249/249 [1:01:31<00:00, 14.82s/it]


In [29]:
import chromadb

In [49]:
client = chromadb.PersistentClient(path="../../../Downloads/chromadb_zip/")

In [50]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mistral",
)

In [35]:
collection = client.get_collection("news_2024_10_22")

In [51]:
collection.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

["You may think you know a person when you meet them once, twice, and even more frequently over a few months. However, it may be possible that you are yet to encounter their darkest secrets, fears, and memories that they themselves are hiding from. Even after knowing someone's stark past, it isn't possible to truly understand the depth of it, however empathetic a person might be.Unknowingly, past trauma can have a big role in your current relationship.Your responses, willingness to develop", "willingness to develop emotional and physical intimacy, and ability to trust can all be affected by demons of your past. It isn't easy to let them go, but with the help of a loving and supporting partner, it's possible to defeat them and emerge victorious.Let's look at how past trauma can impact your current relationships:Emotional triggersWhen a person experiences traumatic events, it can have a significant impact on their current relationship status. If your partner experiences certain", 'experi

In [26]:
vectordb.get()['documents']

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [56]:
results = collection.similarity_search(
    "What is happening in delhi?",
    k=2
)

AttributeError: 'Collection' object has no attribute 'similarity_search'

In [19]:
for result in results:
    print(result.page_content)
    print("#####")

In [21]:
retriever = vectordb.as_retriever(k=4)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""Act as an experienced stock trader with a strong understanding of the Indian stock market. 
    I will provide you with all the relevant news for a specific day from India. Based on the news provided, \
    I need you to perform the following tasks: 
    1. Identify the 5 best potential stocks to buy – these should be companies or sectors expected to perform \
    well based on the news. Provide a brief reason for each stock, explaining why it's a good investment opportunity. 
    2. Identify the 5 worst potential stocks** – these should be companies or sectors expected to face challenges\ 
    or underperform due to the news. Provide a brief reason for each stock, explaining why selling or avoiding this\ 
    stock would be wise. For each stock, mention whether it's related to the sector's performance, company-specific \
    news, or broader economic or political trends, and offer a brief, clear explanation of your rationale. 
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [38]:
template_string = """Translate the text that is delimited by triple backticks \
into a style that is {style}. text: ```{text}```"""

In [39]:
from langchain.prompts import ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_template(template_string)

In [40]:
chat_prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```'), additional_kwargs={})])

In [41]:
customer_style = """American English in a calm and respectful tone"""
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help right now, matey!
"""

In [42]:
request_chatbot = chat_prompt_template.format_messages(style=customer_style, text=customer_email)
request_chatbot

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```", additional_kwargs={}, response_metadata={})]

In [47]:
response = chat.invoke(request_chatbot)

In [48]:
response

'Here is the translated text in American English, written in a calm and respectful tone:\n\n"Hello, I\'m having a bit of an issue with my blender. The lid flew off while I was making a smoothie and it ended up splattering everywhere. To make matters worse, the warranty doesn\'t cover any costs associated with cleaning up the mess. I could really use your help right now."'

In [25]:
len(splits)

1010